# Non-Linear Poisson in 2D

In this section, we consider the non-linear Poisson problem:

$$
-\nabla \cdot \left( (1+u^2) \nabla u \right) = f, \Omega
\\
u = 0, \partial \Omega
$$
where $\Omega$ denotes the unit square.

For testing, we shall take a function $u$ that fulfills the boundary condition, the compute $f$ as

$$
f(x,y) = -\nabla^2 u - F(u)
$$

We will use the Picard Method, which reads in the variational formulation

$$
\mbox{Find } u_{n+1} \in \mathcal{V}_h, \mbox{such that}\\
\int_{\Omega} (1+u_n^2) \nabla u_{n+1} \cdot \nabla v ~ d\Omega = \int_{\Omega} f v ~d\Omega, \quad \forall v \in \mathcal{V}_h
$$

In [1]:
from simplines import compile_kernel, apply_dirichlet

In [2]:
from simplines import SplineSpace
from simplines import TensorSpace
from simplines import StencilMatrix
from simplines import StencilVector

### Example 1

In this example, we take $u(x,y) = \sin(\pi x) \sin(\pi y)$

In [3]:
from gallery_section_08 import assemble_vector_ex01
from gallery_section_08 import assemble_matrix_ex01
from gallery_section_08 import assemble_norm_ex01

In [4]:
assemble_stiffness = compile_kernel(assemble_matrix_ex01, arity=2)
assemble_rhs = compile_kernel(assemble_vector_ex01, arity=1)
# TODO: pyccel is not working here for an unkown reason
assemble_norm_l2 = compile_kernel(assemble_norm_ex01, arity=0, pyccel=False)

In [5]:
from scipy.sparse import csc_matrix, linalg as sla

In [6]:
def picard(V, niter=10, u=None):

    if u is None:
        u   = StencilVector(V.vector_space)

    rhs = assemble_rhs( V )
    rhs = apply_dirichlet(V, rhs)
    b = rhs.toarray()

    for i in range(niter):
        stiffness = assemble_stiffness(V, fields=[u])
        stiffness = apply_dirichlet(V, stiffness)

        M = stiffness.tosparse() 

        lu = sla.splu(csc_matrix(M))    

        x = lu.solve(b)
        x = x.reshape(V.nbasis)

        u.from_array(V, x)
        u = apply_dirichlet(V, u)

        l2_norm = assemble_norm_l2(V, fields=[u])
        print('> iter = {}      l2-norm = {} '.format(i, l2_norm))
        
    return u

In [7]:
# create the spline space for each direction
V1 = SplineSpace(degree=2, nelements=32)
V2 = SplineSpace(degree=2, nelements=32)

# create the tensor space
V = TensorSpace(V1, V2)

In [8]:
u = picard(V, niter=10)

> iter = 0      l2-norm = 0.10416640648161508 
> iter = 1      l2-norm = 0.019793093173538445 
> iter = 2      l2-norm = 0.0032731105024647786 
> iter = 3      l2-norm = 0.00043199279404655954 
> iter = 4      l2-norm = 4.835347275730043e-05 
> iter = 5      l2-norm = 5.705841235569041e-06 
> iter = 6      l2-norm = 3.24981132968314e-06 
> iter = 7      l2-norm = 3.2312370119637365e-06 
> iter = 8      l2-norm = 3.2307034350171563e-06 
> iter = 9      l2-norm = 3.230728747545628e-06 


### Picard method with two-grids

#### Create the fine space

In [9]:
# create the spline space for each direction
V1 = SplineSpace(degree=2, nelements=32)
V2 = SplineSpace(degree=2, nelements=32)

# create the tensor space
Vh = TensorSpace(V1, V2)

#### Create the coarse space

In [10]:
# create the spline space for each direction
V1 = SplineSpace(degree=2, nelements=16)
V2 = SplineSpace(degree=2, nelements=16)

# create the tensor space
VH = TensorSpace(V1, V2)

#### Create the prolongation matrix

In [11]:
from simplines import prolongation_matrix

M = prolongation_matrix(VH, Vh)

#### solve on the coarse space

In [12]:
uH = picard(VH, niter=5)

> iter = 0      l2-norm = 0.10416232003536052 
> iter = 1      l2-norm = 0.01979450032117524 
> iter = 2      l2-norm = 0.0032729508640841334 
> iter = 3      l2-norm = 0.00043360362557916113 
> iter = 4      l2-norm = 5.427312013015149e-05 


#### Transfer the solution from the Coarse to the Fine space

In [13]:
xh = M.dot(uH.toarray())
xh = xh.reshape(Vh.nbasis)

uh = StencilVector(Vh.vector_space)
uh.from_array(Vh, xh)

#### Solve on the fine space

In [14]:
uh = picard(Vh, niter=5, u=uh)

> iter = 0      l2-norm = 5.591139970374466e-06 
> iter = 1      l2-norm = 3.2476216489295532e-06 
> iter = 2      l2-norm = 3.231182998429648e-06 
> iter = 3      l2-norm = 3.2307058132099334e-06 
> iter = 4      l2-norm = 3.230728618015303e-06 
